In [28]:
from backtesting import Backtest,Strategy
import pandas as pd
import numpy as np
import time
import pandas_ta
import ta
from sklearn.neighbors import KNeighborsClassifier
import yfinance as yf

In [29]:
data = pd.read_csv('btc_1h.csv')
data

datetime      open      high       low     close  \
0      2018-01-01 05:30:00  13715.65  13715.65  13400.01  13529.01   
1      2018-01-01 06:30:00  13528.99  13595.89  13155.38  13203.06   
2      2018-01-01 07:30:00  13203.00  13418.43  13200.00  13330.18   
3      2018-01-01 08:30:00  13330.26  13611.27  13290.00  13410.03   
4      2018-01-01 09:30:00  13434.98  13623.29  13322.15  13601.01   
...                    ...       ...       ...       ...       ...   
35659  2022-01-31 01:30:00  37512.37  37740.00  37351.63  37684.45   
35660  2022-01-31 02:30:00  37684.45  37762.60  37492.25  37729.00   
35661  2022-01-31 03:30:00  37729.00  37855.54  37625.41  37785.78   
35662  2022-01-31 04:30:00  37785.78  38230.00  37778.46  37881.76   
35663  2022-01-31 05:30:00  37881.75  37951.07  36828.32  36829.30   

            volume  
0       443.356199  
1       383.697006  
2       429.064572  
3       420.087030  
4       340.807329  
...            ...  
35659  1037.922560  
35660   874.060520  
35661   734.658550  
35662  1728.811800  
35663  2907.710960  

[35664 rows x 6 columns]

In [30]:
data.rename(columns = {'open':'Open','high':'High','low':'Low','close':'Close','volume':'Volume'},inplace=True)

In [31]:
data['datetime'] = pd.to_datetime(data['datetime'])
data.set_index('datetime',inplace=True)
data.rename(columns = {'open':'Open' , 'close':'Close' ,'high' : 'High','low': 'Low'},inplace = True)

In [32]:
def create_features(data):
    
    
    # Calculate RSI
    closing_prices=np.array(data["Close"])
    period = 14
    # Calculate price differences
    deltas = np.diff(closing_prices)
    gains = np.clip(deltas, 0, None)
    losses = np.clip(deltas, None, 0)
    # Create empty lists to store average gains and losses
    avg_gains = []
    avg_losses = []
    # Calculate average gains and losses for each period
    for i in range(len(closing_prices)):
        if i < period - 1:
            avg_gain = np.mean(gains[:i+1])
            avg_loss = -np.mean(losses[:i+1])
        else:
            avg_gain = np.mean(gains[i-period+1:i+1])
            avg_loss = -np.mean(losses[i-period+1:i+1])
        avg_gains.append(avg_gain)
        avg_losses.append(avg_loss)
    # Calculate the RS and RSI values for each period
    rs = [avg_gain / avg_loss for avg_gain, avg_loss in zip(avg_gains, avg_losses)]
    rsi = [100 - (100 / (1 + rsi_val)) for rsi_val in rs]
    data["RSI_14"]=rsi
    
    
    #Calculate Stochastic MACD
    macd_short=12
    macd_long=26
    k_period=14
    data["MACD"] = data['Close'].ewm(span=macd_short).mean() - data['Close'].ewm(span=macd_long).mean()
    lowest_macd = data["MACD"].rolling(window=k_period).min()
    highest_macd = data["MACD"].rolling(window=k_period).max()
    data["Stoch_MACD"] = 100 * (data["MACD"] - lowest_macd) / (highest_macd - lowest_macd)
    data.drop('MACD',axis = 1,inplace=True)
    
    
    #Calculate Stochastic EMA
    k_period = 14
    lowest_low = data["Low"].rolling(window=k_period).min()
    highest_high = data["High"].rolling(window=k_period).max()
    data["Stoch_EMA"] = 100 * (data["Close"] - lowest_low) / (highest_high - lowest_low)
    
    
    #Calculate Stochastic Oscillators
    data.ta.stoch(high='High', low='Low', close='Close', k=14, append=True)
    data.drop('STOCHd_14_3_3',axis = 1,inplace=True)#Remove the %D values
    data["Stoch_OSC"] = data["STOCHk_14_3_3"]
    data.drop('STOCHk_14_3_3',axis = 1,inplace=True)
    
    #Calculate ADX
    adx = ta.trend.ADXIndicator(high=data['High'], low=data['Low'], close=data['Close'], window=14)#Calculate ADX
    data['ADX_14'] = adx.adx()  # Add ADX values as a new column
    
create_features(data)

In [33]:
def Interpolate_Columns(data):
    #Interpolating Missing Values
    data['ADX_14'].interpolate(method='linear',limit_direction='backward',inplace=True)
    data['Stoch_MACD'].interpolate(method='linear',limit_direction='backward',inplace=True)
    data['Stoch_EMA'].interpolate(method='linear',limit_direction='backward',inplace=True)
    data['Stoch_OSC'].interpolate(method='linear',limit_direction='backward',inplace=True)
    
    
Interpolate_Columns(data)

In [34]:
def Generate_Signal(data):
    #Calculation of Composite Indicator
    data['composite'] = np.where(data['ADX_14']<=25,(0.3*(data['RSI_14']+data['Stoch_OSC']) + 0.2*(data['Stoch_EMA'] + data['Stoch_MACD'])),np.where(((data['ADX_14']<=50) & (data['ADX_14']>25)),(0.25*(data['RSI_14']+data['Stoch_OSC']) + 0.25*(data['Stoch_EMA'] + data['Stoch_MACD'])),np.where(((data['ADX_14']<=75) & (data['ADX_14']>50)),(0.225*(data['RSI_14']+data['Stoch_OSC']) + 0.275*(data['Stoch_EMA'] + data['Stoch_MACD'])),np.where(((data['ADX_14']<=100) & (data['ADX_14']>75)),(0.2*(data['RSI_14']+data['Stoch_OSC']) + 0.3*(data['Stoch_EMA'] + data['Stoch_MACD'])),0))))
    #Generating Signal
    data["Signal"]=np.where(((data["ADX_14"]>25)&(data["composite"]<=87)),-1,np.where(((data["ADX_14"]>25)&(data["composite"]>87)),1,np.where(((data["ADX_14"]<=25)&(data["composite"]>87)),1,np.where(((data["ADX_14"]<=25)&(data["composite"]<=87)),-1,0))))

Generate_Signal(data)

In [35]:
data.head()

Open      High       Low     Close      Volume  \
datetime                                                                  
2018-01-01 05:30:00  13715.65  13715.65  13400.01  13529.01  443.356199   
2018-01-01 06:30:00  13528.99  13595.89  13155.38  13203.06  383.697006   
2018-01-01 07:30:00  13203.00  13418.43  13200.00  13330.18  429.064572   
2018-01-01 08:30:00  13330.26  13611.27  13290.00  13410.03  420.087030   
2018-01-01 09:30:00  13434.98  13623.29  13322.15  13601.01  340.807329   

                        RSI_14  Stoch_MACD  Stoch_EMA  Stoch_OSC  ADX_14  \
datetime                                                                   
2018-01-01 05:30:00   0.000000         0.0  20.072793  36.587832     0.0   
2018-01-01 06:30:00  28.057475         0.0  20.072793  36.587832     0.0   
2018-01-01 07:30:00  38.836974         0.0  20.072793  36.587832     0.0   
2018-01-01 08:30:00  54.973063         0.0  20.072793  36.587832     0.0   
2018-01-01 09:30:00  51.957123         0.0  20.072793  36.587832     0.0   

                     composite  Signal  
datetime                                
2018-01-01 05:30:00  14.990908      -1  
2018-01-01 06:30:00  23.408150      -1  
2018-01-01 07:30:00  26.642000      -1  
2018-01-01 08:30:00  31.482827      -1  
2018-01-01 09:30:00  30.578045      -1

In [36]:
data.rename(columns = {'RSI_14':'X_RSI_14' ,'Stoch_MACD':'X_Stoch_MACD','Stoch_EMA':'X_Stoch_EMA','Stoch_OSC':'X_Stoch_OSC','ADX_14':'X_ADX_14','composite':'X_composite','Signal':'X_Signal'},inplace=True)

In [37]:
data.head()

Open      High       Low     Close      Volume  \
datetime                                                                  
2018-01-01 05:30:00  13715.65  13715.65  13400.01  13529.01  443.356199   
2018-01-01 06:30:00  13528.99  13595.89  13155.38  13203.06  383.697006   
2018-01-01 07:30:00  13203.00  13418.43  13200.00  13330.18  429.064572   
2018-01-01 08:30:00  13330.26  13611.27  13290.00  13410.03  420.087030   
2018-01-01 09:30:00  13434.98  13623.29  13322.15  13601.01  340.807329   

                      X_RSI_14  X_Stoch_MACD  X_Stoch_EMA  X_Stoch_OSC  \
datetime                                                                 
2018-01-01 05:30:00   0.000000           0.0    20.072793    36.587832   
2018-01-01 06:30:00  28.057475           0.0    20.072793    36.587832   
2018-01-01 07:30:00  38.836974           0.0    20.072793    36.587832   
2018-01-01 08:30:00  54.973063           0.0    20.072793    36.587832   
2018-01-01 09:30:00  51.957123           0.0    20.072793    36.587832   

                     X_ADX_14  X_composite  X_Signal  
datetime                                              
2018-01-01 05:30:00       0.0    14.990908        -1  
2018-01-01 06:30:00       0.0    23.408150        -1  
2018-01-01 07:30:00       0.0    26.642000        -1  
2018-01-01 08:30:00       0.0    31.482827        -1  
2018-01-01 09:30:00       0.0    30.578045        -1

In [38]:
def get_X(data):
    """Return model design matrix X"""
    return data.filter(like='X').values


def get_y(data):
    """Return dependent variable y"""
    y = data.Close.pct_change(48).shift(-48)  # Returns after roughly two days
    #y[y.between(-.004, .004)] = 0             # Devalue returns smaller than 0.4%
    y[y > 0] = 1
    y[y < 0] = -1
    return y
def get_clean_Xy(data):
    """Return (X, y) cleaned of NaN values"""
    X = get_X(data)
    y = get_y(data).values
    isnan = np.isnan(y)
    X = X[~isnan]
    y = y[~isnan]
    return X, y


In [39]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score 

# Preprocess data ethically, addressing biases and privacy concerns
X, y = get_clean_Xy(data)
# ... (data preprocessing steps)

# Splitting data while ensuring fair representation and avoiding biases
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define hyperparameter grid for tuning, focusing on inclusivity and fairness
param_grid = {'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
              'weights': ['uniform', 'distance'],
              'metric': ['euclidean', 'manhattan']}

# Create grid search object, emphasizing model transparency and explainability
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)

# Fit the grid search to the training set, prioritizing ethical considerations
grid_search.fit(X_train, y_train)

# Retrieve the best model and its hyperparameters
best_knn = grid_search.best_estimator_
print("Best hyperparameters:", grid_search.best_params_)

# Evaluate performance on the testing set, ensuring responsible model usage
y_pred = best_knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)  # Consider diverse evaluation metrics

Best hyperparameters: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}
Accuracy: 0.526530039303762


In [40]:
#Backtesting Strategy 
from backtesting import Backtest , Strategy
from backtesting.lib import TrailingStrategy

N_TRAIN = 500 #Room for optimization


class SignalStrategy(TrailingStrategy):
    price_delta = .004  # 0.4%

    def init(self):
        
        super().init()
        super().set_trailing_sl(4)
        
        # Init our model, a kNN classifier
        self.clf = KNeighborsClassifier(metric='manhattan', n_neighbors=9, weights='uniform')

        # Train the classifier in advance on the first N_TRAIN examples
        df = self.data.df.iloc[:N_TRAIN]
        X, y = get_clean_Xy(df)
        self.clf.fit(X, y)

        # Plot y for inspection
        self.I(get_y, self.data.df, name='y_true')

        # Prepare empty, all-NaN forecast indicator
        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')

    def next(self):
        
        super().next()
        
        # Skip the training, in-sample data
        if len(self.data) < N_TRAIN:
            return

        # Proceed only with out-of-sample data. Prepare some variables
        high, low, close = self.data.High, self.data.Low, self.data.Close
        current_time = self.data.index[-1]

        # Forecast the next movement
        X = get_X(self.data.df.iloc[-1:])
        forecast = self.clf.predict(X)[0]

        # Update the plotted "forecast" indicator
        self.forecasts[-1] = forecast
        
        #Update the signal generated using Technical Analyisi
        current_signal = self.data.X_Signal[-1]

        if forecast == 1 and current_signal == 1:
            if not self.position:
                self.buy()
        
        
        elif forecast == -1 and current_signal==-1:
            if self.position:
                self.position.close()
            
        

In [41]:
bt = Backtest(data, SignalStrategy,cash = 150_000, commission=.0015)
bt.run()

Start                     2018-01-01 05:30:00
End                       2022-01-31 05:30:00
Duration                   1491 days 00:00:00
Exposure Time [%]                    4.376963
Equity Final [$]                934090.388125
Equity Peak [$]                 964655.392105
Return [%]                         522.726925
Buy & Hold Return [%]              172.224649
Return (Ann.) [%]                   56.428342
Volatility (Ann.) [%]               25.530435
Sharpe Ratio                         2.210238
Sortino Ratio                       14.722543
Calmar Ratio                         8.760848
Max. Drawdown [%]                   -6.440968
Avg. Drawdown [%]                   -1.025616
Max. Drawdown Duration      153 days 10:00:00
Avg. Drawdown Duration       10 days 20:00:00
# Trades                                  256
Win Rate [%]                        51.953125
Best Trade [%]                      11.505145
Worst Trade [%]                     -2.899116
Avg. Trade [%]                    